# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,32.56,94,100,4.97,CL,1687730834
1,1,margaret river,-33.9500,115.0667,45.84,84,71,8.70,AU,1687731017
2,2,ushuaia,-54.8000,-68.3000,37.06,81,100,6.91,AR,1687731017
3,3,codrington,-38.2667,141.9667,53.31,80,99,19.17,AU,1687731018
4,4,avarua,-21.2078,-159.7750,80.65,89,40,6.91,CK,1687731018


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_city= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map
map_plot_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_complete = city_data_df.loc[(city_data_df['Max Temp']>= 65) & 
                                         (city_data_df['Max Temp']<=85.6) & 
                                         (city_data_df['Cloudiness']<= 10) & 
                                         (city_data_df['Wind Speed']<=4.5)]

# Drop any rows with null values
city_data_df_complete = city_data_df_complete.dropna()

# Display sample data
city_data_df_complete

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,74,kristiansund,63.1115,7.7320,68.05,59,0,1.14,NO,1687730958
208,208,carrara,44.0629,10.0607,74.75,72,0,1.01,IT,1687731103
230,230,mineiros,-17.5694,-52.5511,68.36,42,0,2.64,BR,1687731113
248,248,brugg,47.4810,8.2087,72.09,80,0,0.16,CH,1687731120
253,253,xunchang,28.4543,104.7150,71.40,86,0,2.28,CN,1687731122
272,272,qianzhou,28.3167,109.7167,70.29,92,0,1.01,CN,1687731130
317,317,santa sylvina,-27.8326,-61.1375,77.47,59,0,3.51,AR,1687731149
345,345,arraial do cabo,-22.9661,-42.0278,71.46,88,0,3.44,BR,1687730964
359,359,ayancik,41.9447,34.5861,72.48,71,0,2.75,TR,1687731166
395,395,taco pozo,-25.6167,-63.2833,74.82,62,0,2.75,AR,1687731181


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df= city_data_df_complete.copy()
hotel_df = hotel_df[["City", 'Country', "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df 

,City,Country,Lat,Lng,Hotel Name
74,kristiansund,NO,63.1115,7.7320,
208,carrara,IT,44.0629,10.0607,
230,mineiros,BR,-17.5694,-52.5511,
248,brugg,CH,47.4810,8.2087,
253,xunchang,CN,28.4543,104.7150,
272,qianzhou,CN,28.3167,109.7167,
317,santa sylvina,AR,-27.8326,-61.1375,
345,arraial do cabo,BR,-22.9661,-42.0278,
359,ayancik,TR,41.9447,34.5861,
395,taco pozo,AR,-25.6167,-63.2833,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [58]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kristiansund - nearest hotel: Comfort Hotel Fosna
carrara - nearest hotel: Bar Trattoria Albergo Maurino
mineiros - nearest hotel: Hotel Dallas
brugg - nearest hotel: Terminus
xunchang - nearest hotel: No hotel found
qianzhou - nearest hotel: JinHuang Hotel
santa sylvina - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
ayancik - nearest hotel: No hotel found
taco pozo - nearest hotel: Hospedaje El Camionero
aquidauana - nearest hotel: Hotel Pantaneiro
oyonnax - nearest hotel: Central Parc Hôtel
nowogard - nearest hotel: Przystań
enshi - nearest hotel: No hotel found
kalaleh - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
74,kristiansund,NO,63.1115,7.7320,Comfort Hotel Fosna
208,carrara,IT,44.0629,10.0607,Bar Trattoria Albergo Maurino
230,mineiros,BR,-17.5694,-52.5511,Hotel Dallas
248,brugg,CH,47.4810,8.2087,Terminus
253,xunchang,CN,28.4543,104.7150,No hotel found
272,qianzhou,CN,28.3167,109.7167,JinHuang Hotel
317,santa sylvina,AR,-27.8326,-61.1375,No hotel found
345,arraial do cabo,BR,-22.9661,-42.0278,No hotel found
359,ayancik,TR,41.9447,34.5861,No hotel found
395,taco pozo,AR,-25.6167,-63.2833,Hospedaje El Camionero


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
map_plot_hotel=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    hover_cols="all",
    use_index=False,
    scale = 0.6,
    color = "City"
)

hvplot.save(map_plot_hotel, 'test.html')
# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)